In [1]:
import sys
sys.path.append("..")

In [2]:
from squlearn.observables import SummedPaulis
from squlearn.qnn import QNNRegressor
from squlearn.qnn.lowlevel_qnn import LowLevelQNN
from squlearn.optimizers import Adam
from squlearn.qnn.loss import ODELoss
from squlearn.qnn.training import *
from squlearn.encoding_circuit import *
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from circuits.circuits import SimpleAnalyticalCircuit

In [3]:
def loss_eq(f_alpha_tensor):
    """
    Args:
        f_array (np.array): array of shape (n, 4) where n is the number of points, each entry corresponds to 
    x, f, dfdx and dfdxdx. This is: f_array = [x, f, dfdx, dfdxdx]
    Returns:
    loss (np.array): array of shape [F] where F is the loss function
    """
    x, f, dfdx = f_alpha_tensor
    return dfdx + f - np.cos(x)

def grad_eq(f_alpha_tensor):
    """
    Args:
        f_array (np.array): array of shape (n, 4) where n is the number of points, each entry corresponds to 
    x, f, dfdx and dfdxdx. This is: f_array = [x, f, dfdx, dfdxdx]
    Returns:
    grad (np.array): array of shape [F, 4] where F is the loss function and 4 is the number of parameters
    """
    x, f, dfdx = f_alpha_tensor
    return np.array([1, 1, 0])

In [4]:
x_line = np.linspace(-0.9, 0.9, 15) #x domain to solve the ODE
x_line = np.array([0.1, 0.2, 0.3, 0.4])
f_initial = [np.cos(x_line[0])] #Initial condition for the ODE

In [5]:
num_qubits = 2
num_features = 1
num_layers = 1
circuit = SimpleAnalyticalCircuit(num_qubits, num_features, num_layers)
#circuit = ChebyshevPQC(num_qubits, num_features, num_layers)
#circuit = HEE
executor = Executor("pennylane")

Observables = SummedPaulis(num_qubits, include_identity=False)                                                      
param_obs = Observables.generate_initial_parameters(seed=1)
param_ini = circuit.generate_initial_parameters(seed=1)

adam = Adam(options={"maxiter": 200, "tol": 0.00009, "log_file": "SHO_DE.log", "skip_mse_fun":False, "lr":0.02})
#loss_harmonic_oscillator, grad_loss_harmonic_oscillator


In [6]:
loss_ODE_np = ODELoss(loss_eq, grad_eq, initial_vec = f_initial, eta=1, boundary_handling = "pinned")

In [7]:
x = sp.symbols('x')
f, dfdx, dfdxdx = sp.symbols("f dfdx dfdxdx")
k = 1
eq = -sp.cos(x) + f + dfdx

In [18]:
y = sp.Function('y')

In [19]:
x = sp.symbols('x')
f, dfdx, dfdxdx = sp.symbols("f dfdx dfdxdx")
k = 1
eq = -sp.cos(x) + sp.exp(f) + dfdx

loss_ODE_sp = ODELoss(eq, initial_vec = f_initial, eta=1, boundary_handling = "pinned", symbols_involved_in_ODE = [x, f, dfdx])

In [20]:
loss_ODE = loss_ODE_sp
ode_regressor = QNNRegressor(
    circuit,
    Observables,
    executor,
    loss_ODE,
    adam,
    param_ini,
    param_obs,
    opt_param_op = True, #Parametrized Observables not benchmarked yet
)


In [21]:
x_line.shape[0]

4

In [22]:
np.exp(1.9488048)

7.020291911444291

In [23]:
ode_regressor.fit(x_line, np.zeros((x_line.shape[0])),  weights=None)
f_QNN = ode_regressor.predict(x_line)

164.11886310313264
(array([0.1, 0.2, 0.3, 0.4]), array([1.91253104, 1.9488048 , 1.97559841, 1.99264417]), array([0.40900746, 0.31586304, 0.21956261, 0.12106839]))
dF dpartial [array([6.77020277, 7.02029189, 7.21093346, 7.33490286]), 1] dimension_of_gradient_with_respect_to dfdp n_param 1
n_param 1
--------------------
grad_envelope_list [[[6.77020277]
  [7.02029189]
  [7.21093346]
  [7.33490286]]

 [[1.        ]
  [1.        ]
  [1.        ]
  [1.        ]]]
(array([0.1, 0.2, 0.3, 0.4]), array([1.91253104, 1.9488048 , 1.97559841, 1.99264417]), array([0.40900746, 0.31586304, 0.21956261, 0.12106839]))
dF dpartial [array([6.77020277, 7.02029189, 7.21093346, 7.33490286]), 1] dimension_of_gradient_with_respect_to dfdop n_param 2
n_param 2
--------------------
grad_envelope_list [[[6.77020277 6.77020277]
  [7.02029189 7.02029189]
  [7.21093346 7.21093346]
  [7.33490286 7.33490286]]

 [[1.         1.        ]
  [1.         1.        ]
  [1.         1.        ]
  [1.         1.        ]]]
149.

KeyboardInterrupt: 

In [ ]:
plt.plot(f_QNN)
plt.plot(f_exact)